In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score # метрика качества  AUC-ROC для оценки принадлежности
# к определенному классу
data = pd.read_csv('data-logistic.csv', header=None)
X = data.iloc[:,1:] # признаки
y = data.iloc[:,0]  # целевой вектор

w1 = 0   # вес 1
w2 = 0   # вес 2
k  = 0.5 # шаг
max_iter = 10000   # максимальное кол-во итераций
evk_min  = 0.00001 # минимальное евклидовое расcтояние
# функция логистической регрессии (по условию задачи)
def logistic_reqression(X, y, w1=0, w2=0, c=0, k=0.1, max_iter=100, evk_min=0.00001):
    l = len(y)
    summa1 = 0
    summa2 = 0
    for j in range(max_iter):
        for i in range(l):
            s = 1 - 1 / (1 + np.exp( -y[i] * (w1 * X[1][i] + w2 * X[2][i]) ))
            summa1 = summa1 + y[i] * X[1][i] * s
            summa2 = summa2 + y[i] * X[2][i] * s
        w1new = w1 + k / l * summa1 - k * c * w1
        w2new = w2 + k / l * summa2 - k * c * w2
        evk = np.sqrt( (w1new - w1)**2 + (w2new - w2)**2 ) # еквклидово расстояние
        if ( evk < evk_min ): break #  условие согласно постановке задачи
        w1, w2 = w1new, w2new   # подбор коэффициентов W1, W2 для минимизации evk
    return w1, w2

def auc_roc(X, y, w1=0, w2=0):
    l = len(y)
    a = []
    for i in range(l):
        a.append( 1 / (1 + np.exp( - w1*X[1][i] - w2*X[2][i])) ) # сигмоидная функция 
        # для расчета оценки вероятностей 
    return roc_auc_score(y, a)

ww1, ww2 = logistic_reqression(X, y, c=0) # вызов функции град.спуска для 
# обычной логистич. регрессии с коэф. регуляризации с=0
print('w1=%.8f, w2=%.8f' %(ww1, ww2))
print(roc_auc_score(y, [1/(1 + np.exp(-ww1*X[1][i] - ww2*X[2][i])) for i in range(len(y))]))

ww1, ww2 = logistic_reqression(X, y, c=10) # вызов функции L2 град.спуска для 
# логистич. регрессии с коэф. регуляризации с=10 
print('w1=%.8f, w2=%.8f' %(ww1, ww2))

print(roc_auc_score(y, [1/(1 + np.exp(-ww1*X[1][i] - ww2*X[2][i])) for i in range(len(y))]))


w1=0.53079847, w2=0.25419886
0.9333333333333331
w1=0.27874229, w2=0.10040298
0.9281904761904761
